In [44]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [46]:
raw_issues = pd.read_json('issues.json')

In [47]:
raw_issues.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,author_association,active_lock_reason,body,closed_by,reactions,timeline_url,performed_via_github_app,state_reason,draft,pull_request
0,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://github.com/vercel/hyper/issues/8041,2562431340,I_kwDOA7enRs6Yu5ls,8041,Allow overriding the default shell on Windows,...,NONE,NaN,**Is your feature request related to a problem...,"{'login': 'jaraco', 'id': 308610, 'node_id': '...",{'url': 'https://api.github.com/repos/vercel/h...,https://api.github.com/repos/vercel/hyper/issu...,NaN,reopened,NaN,NaN
1,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://github.com/vercel/hyper/issues/8040,2562425599,I_kwDOA7enRs6Yu4L_,8040,Support forward slashes on Windows,...,NONE,NaN,**Is your feature request related to a problem...,None,{'url': 'https://api.github.com/repos/vercel/h...,https://api.github.com/repos/vercel/hyper/issu...,NaN,None,NaN,NaN
2,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://api.github.com/repos/vercel/hyper/issu...,https://github.com/vercel/hyper/issues/8039,2562403716,I_kwDOA7enRs6Yuy2E,8039,Support home-relative paths by expanding `~`,...,NONE,NaN,**Is your feature request related to a problem...,None,{'url': 'https://api.github.com/repos/vercel/h...,https://api.github.com/repos/vercel/hyper/issu...,NaN,None,NaN,NaN
3,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel/iss...,https://github.com/vercel/vercel/issues/12281,2586924041,I_kwDOBAnUbs6aMVQJ,12281,`vercel link --repo` very slow,...,NONE,NaN,I used `0x` to trace it and its a glob issue.\...,None,{'url': 'https://api.github.com/repos/vercel/v...,https://api.github.com/repos/vercel/vercel/iss...,NaN,None,NaN,NaN
4,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel/iss...,https://api.github.com/repos/vercel/vercel/iss...,https://github.com/vercel/vercel/pull/12276,2582181376,PR_kwDOBAnUbs5-YZGr,12276,[cli] Add telemetry for `vercel teams invite e...,...,CONTRIBUTOR,NaN,Additional telemetry (and minimal test setup t...,None,{'url': 'https://api.github.com/repos/vercel/v...,https://api.github.com/repos/vercel/vercel/iss...,NaN,None,0.0,{'url': 'https://api.github.com/repos/vercel/v...


In [48]:
raw_issues = raw_issues[['repository_url', 'title', 'body']]

In [58]:
raw_issues.sample(7)

,repo_name,title,body,languages
7,llama-models,Can llama3.2 vision accept no image?,"Hello, \r\n\r\nthank you for thegreat work.\r\...",[Python]
20,stripe-node,Create customer portal session with flow subsc...,### Describe the bug\r\n\r\nTS complains about...,"[TypeScript, JavaScript, Makefile]"
2,hyper,Support home-relative paths by expanding `~`,**Is your feature request related to a problem...,"[TypeScript, Shell, NSIS]"
18,prisma,Prisma client's `.$connect()` does not throw a...,### Bug description\n\nI've encountered this i...,"[TypeScript, JavaScript, Shell]"
4,vercel,[cli] Add telemetry for `vercel teams invite e...,Additional telemetry (and minimal test setup t...,"[TypeScript, JavaScript, HTML]"
6,spring-data-mongodb,Cannot add litteral array with SetUnion,"Hello,\r\n\r\nUsing SetOperators.SetUnion:\r\n...","[Java, Kotlin, Dockerfile]"
16,pandas,"BUG: ""python make.py html"" for documentation i...",### Pandas version checks\r\n\r\n- [X] I have ...,"[Python, Cython, HTML]"


In [50]:
def get_languages(url):
    language_endpoint = url + '/languages'
    response = requests.get(language_endpoint)
    data = response.json()
    langs = list(data.keys())
    res = []
    
    max = 3
    for lang in langs:
        res.append(lang)
        max -= 1
        if max == 0:
            break
    
    return res

raw_issues['languages'] = raw_issues['repository_url'].apply(get_languages)

In [51]:
def extract_repo_name_from_url(url):
    return url.split('/')[-1]

raw_issues['repository_url'] = raw_issues['repository_url'].apply(extract_repo_name_from_url)
raw_issues.rename(columns={'repository_url': 'repo_name'}, inplace=True)
raw_issues.sample(5)

,repo_name,title,body,languages
17,vim-fugitive,:Git command fails with an error `Can't open f...,I recently started experiencing a weird error ...,[Vim Script]
11,pgjdbc,getSchemas with empty string for schemaPattern...,**Describe the issue**\r\nThe method getSchema...,"[Java, Groovy, Shell]"
19,prisma,Inconsistent column data: Could not convert va...,### Bug description\n\nWe recently changed dat...,"[TypeScript, JavaScript, Shell]"
6,spring-data-mongodb,Cannot add litteral array with SetUnion,"Hello,\r\n\r\nUsing SetOperators.SetUnion:\r\n...","[Java, Kotlin, Dockerfile]"
18,prisma,Prisma client's `.$connect()` does not throw a...,### Bug description\n\nI've encountered this i...,"[TypeScript, JavaScript, Shell]"


In [52]:
raw_issues.to_json('cleaned_issues.json')

In [53]:
cleaned_issues = pd.read_json('cleaned_issues.json')
cleaned_issues.sample(5)

,repo_name,title,body,languages
7,llama-models,Can llama3.2 vision accept no image?,"Hello, \r\n\r\nthank you for thegreat work.\r\...",[Python]
16,pandas,"BUG: ""python make.py html"" for documentation i...",### Pandas version checks\r\n\r\n- [X] I have ...,"[Python, Cython, HTML]"
10,commitlint,fix: <title> type-enum inside RuleConfigSeveri...,### Steps to Reproduce\n\n```bash\nimport { Ru...,"[TypeScript, JavaScript]"
11,pgjdbc,getSchemas with empty string for schemaPattern...,**Describe the issue**\r\nThe method getSchema...,"[Java, Groovy, Shell]"
6,spring-data-mongodb,Cannot add litteral array with SetUnion,"Hello,\r\n\r\nUsing SetOperators.SetUnion:\r\n...","[Java, Kotlin, Dockerfile]"


In [54]:
cleaned_issues.isnull().sum()

repo_name    0
title        0
body         0
languages    0
dtype: int64